In [1]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# download the dataset
exist = os.path.isfile("images.zip")
if not exist:
    downloaded = drive.CreateFile({'id': '1WdIdX1zaQPBNkioARqaCkcbyIk_Is_1c'})
    downloaded.GetContentFile('images.zip')
    !unzip images.zip

ModuleNotFoundError: No module named 'google.colab'

In [2]:
%pylab
import cv2
from skimage import measure, draw
from scipy import optimize, ndimage
import matplotlib.pyplot as plt
import imutils
%matplotlib inline
cv2.__version__

Using matplotlib backend: <object object at 0x7f439b043bd0>
Populating the interactive namespace from numpy and matplotlib


'4.10.0'

In [56]:
path = 'Images/complex_background.jpg'

In [58]:
image = cv2.imread('Images/angle_between_len_hand.jpg')
## 第一步，找到蓝色圆形
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
# 定义蓝色的HSV范围
lower_light_blue = np.array([90, 100, 100])
upper_light_blue = np.array([120, 255, 255])

# 创建掩膜
mask = cv2.inRange(hsv, lower_light_blue, upper_light_blue)
# 寻找轮廓
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

## 第二步: 计算手掌与镜头夹角
angle = 0
for contour in contours:
    if cv2.contourArea(contour) > 100:  # 过滤小轮廓
        # 拟合椭圆
        print("yes")
        ellipse = cv2.fitEllipse(contour)
        (x, y), (MA, ma), angle = ellipse
        angle = calculate_angle(ma, MA)

print("手掌与镜头夹角大小: ", angle)

yes
手掌与镜头夹角大小:  48.90801638788893


In [68]:
import math
import torch
import torchvision

"""
该方法需要将蓝色表盘面朝上，与手掌一同拍摄，且手掌部分占据经可能多图片面积
"""

def calculate_angle(major_axis, minor_axis):
    """
    计算手掌和镜头的夹角大小
    :param major_axis: 椭圆的长轴长度
    :param minor_axis: 椭圆的短轴长度
    :return: 夹角（以度为单位）
    """
    eccentricity = math.sqrt(1 - (minor_axis**2 / major_axis**2))
    angle = math.asin(eccentricity)
    angle_degrees = math.degrees(angle)
    return angle_degrees


def output_hand_area(img_path):

    image = cv2.imread(img_path)

    ## 第1步，找到蓝色圆形面积
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # 定义蓝色的HSV范围
    lower_light_blue = np.array([90, 100, 100])
    upper_light_blue = np.array([120, 255, 255])

    # 创建掩膜
    mask = cv2.inRange(hsv, lower_light_blue, upper_light_blue)
    # 寻找轮廓
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    circle_pixel_area = sum(mask != 0)
    print("参照物像素面积: ", circle_pixel_area)

    # ## 第二步: 计算手掌与镜头夹角
    # angle = 0
    # for contour in contours:
    #     if cv2.contourArea(contour) > 100:  # 过滤小轮廓
    #         # 拟合椭圆
    #         ellipse = cv2.fitEllipse(contour)
    #         (x, y), (MA, ma), angle = ellipse
    #         angle = calculate_angle(ma, MA)

    # print("手掌与镜头夹角大小: ", angle)

    ## 第2步: 锐化图片
    kernel = np.array([[-2, -1, 0],
                           [-1, 1, 1],
                           [0, 1, 2]])
    filtered_image = cv2.filter2D(image, -1, kernel)

    ## 第3步: 使用深度学习找到手掌位置
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
    model.eval()

    image_tensor = torchvision.transforms.ToTensor()(image)
    image_tensor = image_tensor.unsqueeze(0)

    # predict segmentation
    with torch.no_grad():
        predictions = model(image_tensor)

    masks = predictions[0]['masks'].detach().cpu().numpy()
    labels = predictions[0]['labels'].cpu().numpy()
    scores = predictions[0]['scores'].cpu().numpy()

    mask = masks[np.where(labels == 1)[0][0], 0]
    # 检查结果
    # masked_image = cv2.bitwise_and(image, image, mask=(mask < 0.5).astype(np.uint8))
    # plt.imshow(masked_image)
    # plt.show()
    hand_pixel_area = sum(mask != 0)
    print("手掌像素点面积: ", hand_pixel_area)

    hand_real = int(7.07 / circle_pixel_area * hand_pixel_area)

    print("手掌真实面积 : ", hand_real)

    return hand_real

In [69]:
path = "Images/angle_between_len_hand.jpg"
output_hand_area(path)

参照物像素面积:  2118
手掌像素点面积:  78153
手掌真实面积 :  260


260